# Configure visualization with AnnData-Zarr

## Motivation

We can configure an interactive visualization that loads the data from the AnnData-Zarr store (the result of the `01_convert_data.ipynb` notebook) using Vitessce.

In [1]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
)
from os.path import join

## Instantiate a `VitessceConfig` object

We begin the configuration process by creating an object using the `VitessceConfig` constructor. This takes three parameters:
- `schema_version`: the schema version that the configuration should conform to. Valid values can be found at http://vitessce.io/docs/view-config-json/#version. The current most recent version `1.0.15` (as of 1/16/2023) is used in this tutorial.
- `name`: a name for the configuration.
- `description` (optional): a brief description of the configuration. This will appear in the `cm.DESCRIPTION` view (if it is added to the configuration).

In [2]:
vc = VitessceConfig(schema_version="1.0.15", name='Habib et al', description='COVID-19 Healthy Donor Brain')

## Add data

To add data to the configuration, we first run [add_dataset](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.add_dataset) which takes the dataset `name` as a parameter.

This returns a new `VitessceConfigDataset` instance. We can add objects which represent local data such as AnnData stores by running [add_object](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfigDataset.add_object) on the dataset instance. To enable multiple `add_object` calls to be chained together, the `add_object` function also returns the `VitessceConfigDataset` instance.

We will store the `VitessceConfigDataset` instance in a variable (`dataset`) to use later when configuring views.

In [3]:
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata_path=join("processed_data", "habib17.zarr"),
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=[["obs/CoarseCellType", "obs/CellType"]],
        obs_set_names=["Cell Type"],
        obs_feature_matrix_path="X"
    )
)

## Add views

Next, we configure the visualization and controller views of interest. Based on the data available in the dataset we added to the configuration above (i.e., the `habib17` AnnData object), we might want to add the following view types:
- a `SCATTERPLOT` view, to visualize the UMAP dimensionality reduction results
- a cell set manager, using the `OBS_SETS` view type (remember that our observations represent cells)
- a list for selecting genes, using the `FEATURE_LIST` view type (remember that our features represent genes)
- a `HEATMAP` view, to visualize the gene expression matrix
- a  cell set sizes bar chart, using the `OBS_SET_SIZES` view type

In [4]:
scatterplot = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
genes = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset)
cell_set_sizes = vc.add_view(vt.OBS_SET_SIZES, dataset=dataset)

## Arrange views

We use the [layout](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.layout) function to specify how the views that we configured above are organized in a grid arrangement. The pipe character `|` and backslash character `/` are "magic" syntax for [horizontal](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.hconcat) and [vertical](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.vconcat) concatenation, respectively.

For more control over the layout, the `x`, `y`, `w` (width), and `h` (height) parameters (not used in this notebook) of the `add_view` function can be used instead of `layout`. Note that the grid has 12 columns and 12 rows, and the x/y/w/h values must be integers.

In [ ]:
vc.layout((scatterplot | cell_sets) / (heatmap | (genes | cell_set_sizes)));

## Render the widget into the notebook

To render the interactive visualization into the notebook, we run the [widget](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.widget) function. 

### What is happening behind the scenes
Because we added local data to the Vitessce configuration (by running `add_object` with an `AnnDataWrapper` object pointing to a local AnnData-Zarr store in `adata_path`), rendering the widget will also result in the local data being served over HTTP on `localhost`. The exact port for the development server can be configured using the `port` parameter of the `widget` function.

In [12]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Habib et al', 'description': 'COVID-19 Healthy Donor Brai…

## Cleanup

While not necessary, you may want to ensure that any local servers are stopped, or you may want to remove the widget from the notebook.

In [10]:
# Remove the widget
vw.close()

In [8]:
from vitessce import data_server

In [11]:
# Stop all running local data servers
data_server.stop_all()